In [20]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt

In [27]:
filename = 'first1000.csv'
df = pd.read_csv(filename,quotechar="\"")
df


,Band,Lyrics,Song
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds
...,...,...,...
994,Judy Garland,"Day in, day out\r\nThe same old hoodoo follows...","Day In, Day Out"
995,Judy Garland,When a dove is in love\r\nWith a doll of a dov...,I Could Go on Singin' ('Til the Cows Come Home)
996,Judy Garland,You've got me where you want me\r\nAnd I hope ...,You Got Me Where You Want Me
997,Judy Garland,"If you're ever in a jam, here I am \r\nIf you'...",Friendship (Medley)


In [32]:
from sentence_transformers import SentenceTransformer


ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

document_embeddings = sbert_model.encode(documents_df['documents_cleaned'])

pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')